# PROGRAM TO REVIEW TRUSS SHOP DRAWINGS - MITEK

In [28]:
'''
IMPORTED RELAVENT MODULES
FOR THIS PROGRAM
'''
import PyPDF2
import os
import re
import openpyxl
from openpyxl.styles import Alignment
from openpyxl.styles import Font
from openpyxl.styles import Border, Side
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule

In [29]:
# TAKING USER INPUT FOR DIFFERENT CRITERIA

design_roof_dead_load = 21
design_floor_dead_load = 18
design_roof_live_load_more_slope = 16
design_roof_live_load_less_slope = 20
design_floor_live_load = 40
actual_truss_spacing = 24
design_wind_standard = 'ASCE 7-16'
design_wind_speed = 115
design_risk_category = 'II'
design_building_code = 'IRC 2018'
design_roof_live_load_deflection_criteria = 360
design_roof_total_load_deflection_criteria = 240
design_floor_live_load_deflection_criteria = 480
design_floor_total_load_deflection_criteria = 360

In [30]:
# LOAD PDF FILE TO THE PROGRAM
pdf_file_name = 'R80335465.1865'
pdf_import = open(pdf_file_name+'.pdf','rb')
pdf = PyPDF2.PdfReader(pdf_import)
page_storage = []


# PARSING THRU ALL PAGES OF THE PDF AND STORING TEXT AS STRING IN A LIST
for page in range(len(pdf.pages)):
    page_storage.append(pdf.pages[page].extract_text())

In [31]:
# PARSING THRU ALL PAGES OF THE PDF AND STORING INFORMATION IN A LIST

# LIST CREATION TO STORE INFORMATION

truss_lable = []
tc_dead_load = []
bc_dead_load = []
tc_live_load = []
bc_live_load = []
slope = []
truss_spacing = []
wind_standard = []
wind_speed = []
risk_category = []
building_code = []
max_tc_stress = []
max_bc_stress = []
max_web_stress = []
actual_live_load_deflection = []
actual_total_load_deflection = []
live_load_deflection_criteria = []
total_load_deflection_criteria = []
drag_load = []
warning = []
truss_type = []
no_of_supports = []
support_reaction_one = []
support_reaction_two = []
support_reaction_three = []
support_reaction_four = []
support_reaction_five = []
total_support_reactions = []


# STORING ALL LISTS IN A DICTONARY. KEYS IN DICTONARY REPRESENTS COLUMNS OF SPREADSHEET

storage_dict = {'A':truss_lable,'B':tc_dead_load,'C':bc_dead_load,'D':tc_live_load,'E':bc_live_load,
                'F':slope,'G':truss_spacing,'H':wind_standard,'I':wind_speed,
                'J':risk_category,'K':building_code,'L':max_tc_stress,'M':max_bc_stress,
                'N':max_web_stress,'O':actual_live_load_deflection,'P':actual_total_load_deflection,
                'Q':live_load_deflection_criteria,'R':total_load_deflection_criteria,'S':drag_load,'T':warning,
                'U':truss_type,'V':no_of_supports,'W':support_reaction_one,'X':support_reaction_two,
                'Y':support_reaction_three,'Z':support_reaction_four,'AA':support_reaction_five,'AB':total_support_reactions}

# SCRIPT TO SET REGULAR EXPRESSION PATTERNS

truss_lable_pattern = re.compile(r'[0-9A-Z]+[0-9-]+[a-zA-Z0-9]*Truss Type')
design_load_pattern = re.compile(r'BCDL[0-9.\n\s*]+')
roof_slope_pattern = re.compile(r'([0-9][.][0-9]{2}) 12[0-9\n]*[a-zA-Z]*')
floor_slope_pattern = re.compile(r'Unbalanced floor live')
page1_pattern = re.compile(r'Page 1')
truss_spacing_pattern = re.compile(r'Code[0-9-]+')
wind_standard_pattern = re.compile(r'Wind:[a-zA-Z0-9\s-]+')
wind_speed_pattern = re.compile(r'Vult=[a-zA-Z0-9]+')
risk_category_pattern = re.compile(r'Cat.\n[A-Z]+')
building_code_pattern = re.compile(r'I[B|R]C[0-9]+')
max_member_stress_pattern = re.compile(r'[\n0-9.]+DEFL')
actual_deflection_pattern = re.compile(r'defl[\n(.*>*\d*|n/r|n/a)]+')
design_deflection_criteria_pattern = re.compile(r'L/d\n[0-9]+\n[0-9]+')
drag_load_pattern = re.compile(r'total drag [a-zA-Z0-9\s]+')
warning_pattern = re.compile(r'WARNING: Required bearing')
truss_type_pattern = re.compile(r'[a-zA-Z]*\s[a-zA-Z-]+Qty')
bearing_support_pattern_t1 = re.compile(r'[0-9]+\s+=\s+[-]*\d+/') # <---- BEARING PATTERN FOR SUPPORT REACTIONS. (lb/size)
bearing_support_pattern_t2 = re.compile(r'Grav\s+([a-zA-Z0-9\s)(=,]+)') # <---- BEARING PATTERN FOR SUPPORT REACTIONS. (size)
cont_support_pattern = re.compile(r'All bearings [0-9-]+')
reaction_pattern_t1 = re.compile(r'[-]*\d+')                     # <---- PATTERN TO IDENTIFY SUPPORT REACTIONS. (lb/size)
reaction_pattern_t2 = re.compile(r'\d=(\d+)')                     # <---- PATTERN TO IDENTIFY SUPPORT REACTIONS. (size)
reaction_check_pattern_1 = re.compile(r'REACTIONS.\s+[(][a-z/]*[)]') # <---- PATTERN TO CATCH REACTIONS. (lb/size) or (size)
reaction_check_pattern_2 = re.compile(r'REACTIONS. All bearings') # <---- PATTERN TO CATCH REACTIONS. All bearings [THIS TYPE OF PATTERN DOES NOT HAVE (lb/size) or (size)]
truss_layout_pattern = re.compile('PLACEMENT DIAGRAM') # <---- PATTERN TO IDENTIFY IF THE PAGE IS A LAYOUT PAGE OR NOT

# PARSING THRU ALL PAGES OF THE PDF AND CAPTURING IMPORTANT INFORMATION AND STORING IT IN THE LIST
# ENUMERATE HAS BEEN USED TO ALSO OBTAIN THE INDEX OF THAT PAGE. THE INDEX OBTAINED IS THEN USED TO CATCH DRAG LOAD ON PAGE 2.
# THIS WAY THE PROGRAM QUICKLY CHECKS THE VERY NEXT PAGE FOR ANY DRAG LOADS PRESENT AND IF NOT FOUND THEN ADDS A "-" TO THE LIST

for index, page in enumerate(page_storage):

# SETTING CONTROL STATEMENTS BASED ON THREE CONDITIONS:
# 1. TRUSS LABEL NEEDS TO BE PRESENT ON A PAGE ALONG WITH 2. NEEDS TO BE ON PAGE {1} 3. SECONDARY CRITERIA TO BE CAPTURED
# THIS WILL ENSURE THAT THE SECONDARY CRITERIA CAPTURED ACTUALLY BELONGS TO THE SAME PAGE AS THE TRUSS

# SCRIPT TO READ TRUSS LABLE
    lable_found = re.findall(truss_lable_pattern,page)
    page1_pattern_found = re.findall(page1_pattern,page)
    # CONTROL STATEMENT TO VERIFY IF THE PAGE IS A LAYOUT PAGE OR NOT.
    # LOGIC FILTERS OUT THE PAGES THAT ARE LAYOUT PAGES.
    truss_layout_pattern_found = re.findall(truss_layout_pattern,page)
    if len(lable_found) != 0 and len(page1_pattern_found) != 0 and len(truss_layout_pattern_found) == 0:
        truss_lable.append(lable_found[0].split('Truss')[0])

# SCRIPT TO READ DESIGN LOADS
    design_load_found = re.findall(design_load_pattern,page)
    # CONTROL STATEMENT TO FIND TRUSS LABEL AND SECONDARY CRITERIA ON PAGE 1
    # THIS CONTROL STATEMENT GETS REPLICATED FOR SECONDARY CRITERIA BELOW
    page1_pattern_found = re.findall(page1_pattern,page)
    if len(design_load_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) != 0:
        tc_dead_load.append(design_load_found[0].split('\n')[1].strip('BCDL *'))
        bc_dead_load.append(design_load_found[0].split('\n')[3].strip('BCDL *'))
        tc_live_load.append(design_load_found[0].split('\n')[0].strip('BCDL *'))
        bc_live_load.append(design_load_found[0].split('\n')[2].strip('BCDL *'))

# SCRIPT TO READ SLOPE
    roof_slope_found = re.findall(roof_slope_pattern,page)
    floor_slope_found = re.findall(floor_slope_pattern,page) # REGEX TO IDENTIFY IF THE TRUSS IS ROOF OR FLOOR
    page1_pattern_found = re.findall(page1_pattern,page)
    if len(roof_slope_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) != 0:
        slope.append(roof_slope_found[0])
    elif len(roof_slope_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) != 0:
        slope.append('0.00')
    elif len(floor_slope_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        slope.append('0.00')
    elif len(floor_slope_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        slope.append('0.00')

# SCRIPT TO READ TRUSS SPACING
    truss_spacing_found = re.findall(truss_spacing_pattern,page)
    if len(truss_spacing_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) != 0:
        truss_spacing.append(truss_spacing_found[0].split('-')[0].strip('Code'))

# SCRIPT TO READ WIND STANDARD
    wind_standard_found = re.findall(wind_standard_pattern,page)
    if len(wind_standard_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        wind_standard.append(wind_standard_found[0].strip('Wind: '))

    elif len(wind_standard_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        wind_standard.append('NA')

# SCRIPT TO READ WIND SPEED
    wind_speed_found = re.findall(wind_speed_pattern,page)
    if len(wind_speed_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        wind_speed.append(wind_speed_found[0].strip('Vult= mph'))

    elif len(wind_speed_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        wind_speed.append('NA')

# SCRIPT TO READ RISK CATEGORY
    risk_category_found = re.findall(risk_category_pattern,page)
    if len(risk_category_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        risk_category.append(risk_category_found[0].strip('Cat.\n'))

    elif len(risk_category_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        risk_category.append('NA')

# SCRIPT TO READ DESIGN BUILDING CODE
    building_code_found = re.findall(building_code_pattern,page)
    if len(building_code_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        code = ''
        year = ''
        for i in building_code_found[0]:
            if re.search(r'[A-Z]',i):
                code += i
            elif re.search(r'[0-9]',i):
                year += i
        code_year = code+' '+year
        building_code.append(code_year)

# SCRIPT TO READ MAX MEMBER STRESS
    max_member_stress_found = re.findall(max_member_stress_pattern,page)
    if len(max_member_stress_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        max_tc_stress.append(max_member_stress_found[0].split('\n')[0].strip('DEFL'))
        max_bc_stress.append(max_member_stress_found[0].split('\n')[1].strip('DEFL'))
        max_web_stress.append(max_member_stress_found[0].split('\n')[2].strip('DEFL'))

# SCRIPT TO READ DEFLECTION CRITERIA
    actual_deflection_found = re.findall(actual_deflection_pattern,page)
    design_deflection_criteria_found = re.findall(design_deflection_criteria_pattern,page)

    if len(design_deflection_criteria_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        live_load_deflection_criteria.append(design_deflection_criteria_found[0].split('\n')[1])
        total_load_deflection_criteria.append(design_deflection_criteria_found[0].split('\n')[2])

    elif len(design_deflection_criteria_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        live_load_deflection_criteria.append('NA')
        total_load_deflection_criteria.append('NA')

    if len(actual_deflection_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        actual_live_load_deflection.append(actual_deflection_found[0].split('\n')[1].strip('><'))
        actual_total_load_deflection.append(actual_deflection_found[0].split('\n')[2].strip('><'))

# SCRIPT TO READ DRAG LOAD
    drag_load_found = re.findall(drag_load_pattern,page)
    if len(drag_load_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        for load in drag_load_found[0].split(' '):
            find_drag_pattern = re.compile(r'[0-9]+')
            find_drag_pattern_found = re.findall(find_drag_pattern,load)
            if find_drag_pattern_found != []:
                drag_load.append(find_drag_pattern_found[0])

    # IF NO DRAG LOAD IS FOUND ON PAGE 1, THEN THIS PORTION OF THE CODE CHECKS IT ON PAGE 2

    elif len(drag_load_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:

        if index+1 < len(page_storage):

            drag_load_found = re.findall(drag_load_pattern,page_storage[index+1]) # THIS LINE CHECKS FOR ANY DRAG LOAD PRESENT ON THE VERY NEXT PAGE (THAT IS PAGE 2)
            page2_pattern = re.compile(r'Page 2')                                 # REGULAR EXPRESSION PATTERN TO IDENTIFY PAGE 2
            page2_pattern_found = re.findall(page2_pattern,page_storage[index+1]) # THIS LINE CHECKS IF THE PAGE IS PAGE 2 OR NOT

            if len(drag_load_found) != 0 and len(lable_found) != 0 and len(page2_pattern_found) !=0: # THIS LINE CHECKS IF THE DRAG LOAD FOUND IS ON PAGE 2 OF THE SAME TRUSS. 
                                                                                                    # THAT WAY IT DOES NOT SKIP OVER TO A NEW TRUSS
                for load in drag_load_found[0].split(' '):
                    find_drag_pattern = re.compile(r'[0-9]+')
                    find_drag_pattern_found = re.findall(find_drag_pattern,load)
                    if find_drag_pattern_found != []:
                        drag_load.append(find_drag_pattern_found[0])
            else:                                                     # IF NO DRAG LOAD IS FOUND EVEN ON THE SECOND PAGE, THEN IT INSERTS "-" IN THE DRAG LOAD LIST
                drag_load.append('-')
        
        else:                                                         # MAKES SURE IT ADDS "-" ON THE VERY LAST PAGE IF NO DRAG LOAD IS FOUND
                drag_load.append('-')

# SCRIPT TO READ WARNING
    warning_found = re.findall(warning_pattern,page)
    if len(warning_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        warning.append(warning_found[0].strip(': Required bearing'))
    elif len(warning_found) == 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        warning.append('-')

# SCRIPT TO READ TRUSS TYPE
    truss_type_pattern_found = re.findall(truss_type_pattern,page)
    if len(truss_type_pattern_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        truss_type.append(re.sub('Qty','',truss_type_pattern_found[0]))

# SCRIPT TO READ SUPPORT CONDITION AND SUPPORT REACTIONS AND ADDING THEM TOGETHER
        
    reaction_check_pattern_1_found = re.findall(reaction_check_pattern_1,page)
    reaction_check_pattern_2_found = re.findall(reaction_check_pattern_2,page)
    cont_support_found = re.findall(cont_support_pattern,page)

    # CONDITIONAL STATEMENTS TO FOR PDF HAVING REACTIONS. (lb/size)

    if len(reaction_check_pattern_1_found) != 0 and reaction_check_pattern_1_found[0].split()[1] == '(lb/size)':

        bearing_support_found = re.findall(bearing_support_pattern_t1,page)

        if len(bearing_support_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
            no_of_supports.append(str(len(bearing_support_found)))
            if len(bearing_support_found) == 2:
                reaction_pattern_found_one = re.findall(reaction_pattern_t1, bearing_support_found[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t1, bearing_support_found[1])
                support_reaction_one.append(reaction_pattern_found_one[1])
                support_reaction_two.append(reaction_pattern_found_two[1])
                support_reaction_three.append('-')
                support_reaction_four.append('-')
                support_reaction_five.append('-')
                total_support_reactions.append(str(float(reaction_pattern_found_one[1]) + float(reaction_pattern_found_two[1])))

            elif len(bearing_support_found) == 3:
                reaction_pattern_found_one = re.findall(reaction_pattern_t1, bearing_support_found[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t1, bearing_support_found[1])
                reaction_pattern_found_three = re.findall(reaction_pattern_t1, bearing_support_found[2])
                support_reaction_one.append(reaction_pattern_found_one[1])
                support_reaction_two.append(reaction_pattern_found_two[1])
                support_reaction_three.append(reaction_pattern_found_three[1])
                support_reaction_four.append('-')
                support_reaction_five.append('-')
                total_support_reactions.append(str(float(reaction_pattern_found_one[1]) + float(reaction_pattern_found_two[1]) + 
                                               float(reaction_pattern_found_three[1])))
                

            elif len(bearing_support_found) == 4:
                reaction_pattern_found_one = re.findall(reaction_pattern_t1, bearing_support_found[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t1, bearing_support_found[1])
                reaction_pattern_found_three = re.findall(reaction_pattern_t1, bearing_support_found[2])
                reaction_pattern_found_four = re.findall(reaction_pattern_t1, bearing_support_found[3])
                support_reaction_one.append(reaction_pattern_found_one[1])
                support_reaction_two.append(reaction_pattern_found_two[1])
                support_reaction_three.append(reaction_pattern_found_three[1])
                support_reaction_four.append(reaction_pattern_found_four[1])
                support_reaction_five.append('-')
                total_support_reactions.append(str(float(reaction_pattern_found_one[1]) + float(reaction_pattern_found_two[1]) + 
                                               float(reaction_pattern_found_three[1]) + float(reaction_pattern_found_four[1])))

            elif len(bearing_support_found) == 5:
                reaction_pattern_found_one = re.findall(reaction_pattern_t1, bearing_support_found[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t1, bearing_support_found[1])
                reaction_pattern_found_three = re.findall(reaction_pattern_t1, bearing_support_found[2])
                reaction_pattern_found_four = re.findall(reaction_pattern_t1, bearing_support_found[3])
                reaction_pattern_found_five = re.findall(reaction_pattern_t1, bearing_support_found[4])
                support_reaction_one.append(reaction_pattern_found_one[1])
                support_reaction_two.append(reaction_pattern_found_two[1])
                support_reaction_three.append(reaction_pattern_found_three[1])
                support_reaction_four.append(reaction_pattern_found_four[1])
                support_reaction_five.append(reaction_pattern_found_five[1])
                total_support_reactions.append(str(float(reaction_pattern_found_one[1]) + float(reaction_pattern_found_two[1]) + 
                                               float(reaction_pattern_found_three[1]) + float(reaction_pattern_found_four[1]) +
                                               float(reaction_pattern_found_five[1])))

    # CONDITIONAL STATEMENTS TO FOR PDF HAVING REACTIONS. (size)

    if len(reaction_check_pattern_1_found) != 0 and reaction_check_pattern_1_found[0].split()[1] == '(size)':
        
        bearing_support_found = re.findall(bearing_support_pattern_t2,page)

        if len(bearing_support_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
            no_of_supports.append(str(len(bearing_support_found[0].split(','))))
            if len(bearing_support_found[0].split(',')) == 2:
                reaction_pattern_found_one = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[1])
                support_reaction_one.append(reaction_pattern_found_one[0])
                support_reaction_two.append(reaction_pattern_found_two[0])
                support_reaction_three.append('-')
                support_reaction_four.append('-')
                support_reaction_five.append('-')
                total_support_reactions.append(str(float(reaction_pattern_found_one[0]) + float(reaction_pattern_found_two[0])))


            elif len(bearing_support_found[0].split(',')) == 3:
                reaction_pattern_found_one = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[1])
                reaction_pattern_found_three = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[2])
                support_reaction_one.append(reaction_pattern_found_one[0])
                support_reaction_two.append(reaction_pattern_found_two[0])
                support_reaction_three.append(reaction_pattern_found_three[0])
                support_reaction_four.append('-')
                support_reaction_five.append('-')
                total_support_reactions.append(str(float(reaction_pattern_found_one[0]) + float(reaction_pattern_found_two[0]) + 
                                               float(reaction_pattern_found_three[0])))


            elif len(bearing_support_found[0].split(',')) == 4:
                reaction_pattern_found_one = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[2])
                reaction_pattern_found_three = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[3])
                reaction_pattern_found_four = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[4])
                support_reaction_one.append(reaction_pattern_found_one[0])
                support_reaction_two.append(reaction_pattern_found_two[0])
                support_reaction_three.append(reaction_pattern_found_three[0])
                support_reaction_four.append(reaction_pattern_found_four[0])
                support_reaction_five.append('-')
                total_support_reactions.append(str(float(reaction_pattern_found_one[0]) + float(reaction_pattern_found_two[0]) + 
                                               float(reaction_pattern_found_three[0]) + float(reaction_pattern_found_four[0])))

            elif len(bearing_support_found[0].split(',')) == 5:
                reaction_pattern_found_one = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[0])
                reaction_pattern_found_two = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[1])
                reaction_pattern_found_three = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[2])
                reaction_pattern_found_four = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[3])
                reaction_pattern_found_five = re.findall(reaction_pattern_t2, bearing_support_found[0].split(',')[4])
                support_reaction_one.append(reaction_pattern_found_one[0])
                support_reaction_two.append(reaction_pattern_found_two[0])
                support_reaction_three.append(reaction_pattern_found_three[0])
                support_reaction_four.append(reaction_pattern_found_four[0])
                support_reaction_five.append(reaction_pattern_found_five[0])
                total_support_reactions.append(str(float(reaction_pattern_found_one[0]) + float(reaction_pattern_found_two[0]) + 
                                               float(reaction_pattern_found_three[0]) + float(reaction_pattern_found_four[0]) +
                                               float(reaction_pattern_found_five[0])))


    elif len(cont_support_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        no_of_supports.append('CONT.')
        support_reaction_one.append('-')
        support_reaction_two.append('-')
        support_reaction_three.append('-')
        support_reaction_four.append('-')
        support_reaction_five.append('-')
        total_support_reactions.append('0')

    elif len(reaction_check_pattern_2_found) != 0 and len(lable_found) != 0 and len(page1_pattern_found) !=0:
        no_of_supports.append('CONT.')
        support_reaction_one.append('-')
        support_reaction_two.append('-')
        support_reaction_three.append('-')
        support_reaction_four.append('-')
        support_reaction_five.append('-')
        total_support_reactions.append('0')

In [32]:
# CREATING A WORKBOOK AND SAVING IT IN THE CURRENT WORKING DIRECTORY

wb = openpyxl.Workbook()
ws = wb.active
file_name = 'Truss Review Results_'+pdf_file_name
ws.title = "Summary"
directory_path = os.getcwd()
file_path = directory_path+'/'+file_name+'.xlsx'

In [33]:
'''
storage_dict = {'A':truss_lable,'B':tc_dead_load,'C':bc_dead_load,'D':tc_live_load,'E':bc_live_load,
                'F':slope,'G':truss_spacing,'H':wind_standard,'I':wind_speed,
                'J':risk_category,'K':building_code,'L':max_tc_stress,'M':max_bc_stress,
                'N':max_web_stress,'O':actual_live_load_deflection,'P':actual_total_load_deflection,
                'Q':live_load_deflection_criteria,'R':total_load_deflection_criteria,'S':drag_load,'T':warning,
                'U':truss_type,'V':no_of_supports,'W':support_reaction_one,'X':support_reaction_two,
                'Y':support_reaction_three,'Z':support_reaction_four,'AA':support_reaction_five,'AB':total_support_reactions}
'''


# INITTIALIZING THE TITLE ROW

ws['A1'] = 'TRUSS LABEL'
ws['B1'] = 'TOP CHORD DEAD LOAD'
ws['C1'] = 'BOTTOM CHORD DEAD LOAD'
ws['D1'] = 'TOP CHORD LIVE LOAD'
ws['E1'] = 'BOTTOM CHORD LIVE LOAD'
ws['F1'] = 'SLOPE'
ws['G1'] = 'TRUSS SPACING'
ws['H1'] = 'WIND STANDARD'
ws['I1'] = 'WIND SPEED'
ws['J1'] = 'RISK CATEGORY'
ws['K1'] = 'BUILDING CODE'
ws['L1'] = 'MAX TOP CHORD STRESS'
ws['M1'] = 'MAX BOTTOM CHORD STRESS'
ws['N1'] = 'MAX WEB CHORD STRESS'
ws['O1'] = 'CALCULATED LIVE LOAD DEFLECTION'
ws['P1'] = 'CALCULATED TOTAL LOAD DEFLECTION'
ws['Q1'] = 'LIVE LOAD DEFLECTION CRITERIA'
ws['R1'] = 'TOTAL LOAD DEFLECTION CRITERIA'
ws['S1'] = 'DRAG LOAD'
ws['T1'] = 'WARNING'
ws['U1'] = 'TRUSS TYPE'
ws['V1'] = 'NO. OF SUPPORTS'
ws['W1'] = 'REACTION AT SUPPORT 1'
ws['X1'] = 'REACTION AT SUPPORT 2'
ws['Y1'] = 'REACTION AT SUPPORT 3'
ws['Z1'] = 'REACTION AT SUPPORT 4'
ws['AA1'] = 'REACTION AT SUPPORT 5'
ws['AB1'] = 'TOTAL SUPPORT REACTIONS'


# INITTIALIZING THE UNITS ROW

ws['B2'] = '(PSF)'
ws['C2'] = '(PSF)'
ws['D2'] = '(PSF)'
ws['E2'] = '(PSF)'
ws['F2'] = '( /12)'
ws['G2'] = '(FT)'
ws['H2'] = '(MPH)'
ws['O2'] = '(L/)'
ws['P2'] = '(L/)'
ws['Q2'] = '(L/)'
ws['R2'] = '(L/)'
ws['S2'] = '(LB)'
ws['V2'] = '(#)'
ws['W2'] = '(LB)'
ws['X2'] = '(LB)'
ws['Y2'] = '(LB)'
ws['Z2'] = '(LB)'
ws['AA2'] = '(LB)'
ws['AB2'] = '(LB)'


# SETTING UP ROW AND COLUMN SIZE

ws.row_dimensions[1].height = 70
ws.row_dimensions[2].height = 15
ws.freeze_panes = 'B3'

for column in storage_dict.keys():

    # SETTING CELL WIDTH
    ws.column_dimensions[column].width = 11

    # SETTING FONT ALIGNMENT AND FONT STYLE
    ws[f'{column}1'].alignment = Alignment(horizontal = 'center',vertical = 'center',wrap_text = 'True')
    ws[f'{column}2'].alignment = Alignment(horizontal='center',vertical = 'center')
    ws[f'{column}1'].font = Font(bold = 'True')
    ws[f'{column}2'].font = Font(bold = 'True')

    # SETTING CELL BORDER STYLE
    thin_border = Side(border_style = 'thin', color = '000000')
    border_format = Border(bottom = thin_border)
    ws[f'{column}2'].border = border_format

# SETTING UP CELL BORDER FOR FIRST COLUMN ONLY : AT TRUSS LABELS

for i in range(len(storage_dict['A'])):
    thin_border = Side(border_style = 'thin', color='000000')
    border_format = Border(right = thin_border)
    ws[f'A{i+4}'].border = border_format

In [34]:
# UNPACKING DICTONARY AND STORING VALUES IN CELLS

type_non_float = re.compile(r'[*a-zA-Z-]+')

for column in storage_dict.keys():
    for row_num in range(len(storage_dict[column])):
        if re.search(type_non_float,storage_dict[column][row_num]):
            ws[f'{column}{4+row_num}'] = storage_dict[column][row_num]
            ws[f'{column}{4+row_num}'].alignment = Alignment(horizontal='center',vertical = 'center')
        else:
            ws[f'{column}{4+row_num}'] = float(storage_dict[column][row_num])
            ws[f'{column}{4+row_num}'].alignment = Alignment(horizontal='center',vertical = 'center')

In [35]:
# PARSING THRU ALL COLUMNS AND CHECKING IF IT MATCHES USER SPECIFIED CRITERIA
# IF VALUES IN THE CELLS DO NOT MEET USER SPECIFIED MINIMUM CRITERIS, THEN IT WOULD BE HIGHLIGHTED

# HIGHLIGHT CONDITIONS
# FOR MORE COLORS, VISIT: https://www.rapidtables.com/web/color/RGB_Color.html

font_color = '00FF0000'
cell_color = '00FFCC99'
bold_condition = True
filltype = 'solid'

for i in range(len(storage_dict['A'])):

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING DESIGN DEAD LOAD CONDITION

    # CHECKING ROOF DEAD LOAD CONDITION. HAVING A CONTROL STATEMENT TO CHECK TRUSS SLOPE
    # ROOF TRUSS WILL HAVE SOME SLOPE AND FLOOR TRUSS WILL HAVE ZERO SLOPE
    if float(storage_dict['F'][i]) != 0:
        if eval(storage_dict['B'][i]+'+'+storage_dict['C'][i]) < design_roof_dead_load:
            ws[f'B{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'C{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'B{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)
            ws[f'C{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        else:
            ws[f'B{i+4}'].font = Font(color = None,bold = None)
            ws[f'C{i+4}'].font = Font(color = None,bold = None)
            ws[f'B{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)
            ws[f'C{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    # CHECKING FLOOR DEAD LOAD CONDITION. HAVING A CONTROL STATEMENT TO CHECK TRUSS SLOPE
    # ROOF TRUSS WILL HAVE SOME SLOPE AND FLOOR TRUSS WILL HAVE ZERO SLOPE
    if float(storage_dict['F'][i]) == 0:
        if eval(storage_dict['B'][i]+'+'+storage_dict['C'][i]) < design_floor_dead_load:
            ws[f'B{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'C{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'B{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)
            ws[f'C{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        else:
            ws[f'B{i+4}'].font = Font(color = None,bold = None)
            ws[f'C{i+4}'].font = Font(color = None,bold = None)
            ws[f'B{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)
            ws[f'C{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING DESIGN LIVE LOAD CONDITION WITH RESPECT TO SLOPE

    # CHECKING LIVE LOAD FOR HIGHER ROOF SLOPE CONDITION
    if float(storage_dict['F'][i]) >= 4:
        if eval(storage_dict['D'][i]+'+'+storage_dict['E'][i]) < design_roof_live_load_more_slope:
            ws[f'D{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'E{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'D{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)
            ws[f'E{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        elif eval(storage_dict['D'][i]+'+'+storage_dict['E'][i]) >= design_roof_live_load_more_slope:
            ws[f'D{i+4}'].font = Font(color = None,bold = None)
            ws[f'E{i+4}'].font = Font(color = None,bold = None)
            ws[f'D{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)
            ws[f'E{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    # CHECKING LIVE LOAD FOR LOWER ROOF SLOPE CONDITION
    elif float(storage_dict['F'][i]) < 4 and float(storage_dict['F'][i]) != 0:
        if eval(storage_dict['D'][i]+'+'+storage_dict['E'][i]) < design_roof_live_load_less_slope:
            ws[f'D{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'E{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'D{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)
            ws[f'E{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        elif eval(storage_dict['D'][i]+'+'+storage_dict['E'][i]) >= design_roof_live_load_less_slope:
            ws[f'D{i+4}'].font = Font(color = None,bold = None)
            ws[f'E{i+4}'].font = Font(color = None,bold = None)
            ws[f'D{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)
            ws[f'E{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    # CHECKING LIVE LOAD FOR FLOOR SLOPE CONDITION. FLOOR SLOPE HAS ZERO SLOPE
    elif float(storage_dict['F'][i]) == 0:
        if eval(storage_dict['D'][i]+'+'+storage_dict['E'][i]) < design_floor_live_load:
            ws[f'D{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'E{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'D{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)
            ws[f'E{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        elif eval(storage_dict['D'][i]+'+'+storage_dict['E'][i]) >= design_floor_live_load:
            ws[f'D{i+4}'].font = Font(color = None,bold = None)
            ws[f'E{i+4}'].font = Font(color = None,bold = None)
            ws[f'D{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)
            ws[f'E{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING TRUSS SPACING
    if float(storage_dict['G'][i])/12 > actual_truss_spacing:
        ws[f'G{i+4}'].font = Font(color = font_color,bold = bold_condition)
        ws[f'G{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['G'][i])/12 <= actual_truss_spacing:
        ws[f'G{i+4}'].font = Font(color = None,bold = None)
        ws[f'G{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING DESIGN WIND CODE

    if float(storage_dict['F'][i]) != 0:
        if storage_dict['H'][i] != design_wind_standard:
            ws[f'H{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'H{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        elif storage_dict['H'][i] == design_wind_standard:
            ws[f'H{i+4}'].font = Font(color = None,bold = None)
            ws[f'H{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    elif float(storage_dict['F'][i]) == 0:
            ws[f'H{i+4}'].font = Font(color = None,bold = None)
            ws[f'H{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING DESIGN WIND SPEED
    if storage_dict['I'][i] != 'NA' and float(storage_dict['F'][i]) != 0:
        if float(storage_dict['I'][i]) < design_wind_speed:
            ws[f'I{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'I{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        elif float(storage_dict['I'][i]) >= design_wind_speed:
            ws[f'I{i+4}'].font = Font(color = None,bold = None)
            ws[f'I{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    elif storage_dict['I'][i] == 'NA' and float(storage_dict['F'][i]) == 0:
            ws[f'I{i+4}'].font = Font(color = None,bold = None)
            ws[f'I{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    elif storage_dict['I'][i] == 'NA' and float(storage_dict['F'][i]) != 0:
            ws[f'I{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'I{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING RISK CATEGORY
    if storage_dict['J'][i] != 'NA' and storage_dict['F'][i] != 0:
      if storage_dict['J'][i] != design_risk_category:
          ws[f'J{i+4}'].font = Font(color = font_color,bold = bold_condition)
          ws[f'J{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

      elif storage_dict['J'][i] == design_risk_category:
          ws[f'J{i+4}'].font = Font(color = None,bold = None)
          ws[f'J{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    if storage_dict['J'][i] == 'NA' and float(storage_dict['F'][i]) != 0:
      ws[f'J{i+4}'].font = Font(color = font_color,bold = bold_condition)
      ws[f'J{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif storage_dict['J'][i] == 'NA' and float(storage_dict['F'][i]) == 0:
      ws[f'J{i+4}'].font = Font(color = None,bold = None)
      ws[f'J{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING DESIGN BUILDING CODE
    if storage_dict['K'][i] != design_building_code:
        ws[f'K{i+4}'].font = Font(color = font_color,bold = bold_condition)
        ws[f'K{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif storage_dict['K'][i] == design_building_code:
        ws[f'K{i+4}'].font = Font(color = None,bold = None)
        ws[f'K{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING TOP CHORD MEMBER STRESS
    if float(storage_dict['L'][i]) >= 1:
        ws[f'L{i+4}'].font = Font(color = font_color,bold = bold_condition)
        ws[f'L{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['L'][i]) < 1:
        ws[f'L{i+4}'].font = Font(color = None,bold = None)
        ws[f'L{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING BOTTOM CHORD MEMBER STRESS
    if float(storage_dict['M'][i]) >= 1:
        ws[f'M{i+4}'].font = Font(color = font_color,bold = bold_condition)
        ws[f'M{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['M'][i]) < 1:
        ws[f'M{i+4}'].font = Font(color = None,bold = None)
        ws[f'M{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING WEB MEMBER STRESS
    if float(storage_dict['N'][i]) >= 1:
        ws[f'N{i+4}'].font = Font(color = font_color,bold = bold_condition)
        ws[f'N{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['N'][i]) < 1:
        ws[f'N{i+4}'].font = Font(color = None,bold = None)
        ws[f'N{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING LIVE LOAD DEFLECTION

    list_of_not_applicable = ['NA','n/a','n/r','****']
    if storage_dict['O'][i] not in list_of_not_applicable:
        if float(storage_dict['O'][i]) < float(storage_dict['Q'][i]):
            ws[f'O{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'O{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        elif float(storage_dict['O'][i]) >= float(storage_dict['Q'][i]):
            ws[f'O{i+4}'].font = Font(color = None,bold = None)
            ws[f'O{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    elif storage_dict['O'][i] in list_of_not_applicable:
      ws[f'O{i+4}'].font = Font(color = None,bold = None)
      ws[f'O{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING TOTAL LOAD DEFLECTION
    if storage_dict['P'][i] not in list_of_not_applicable:
        if float(storage_dict['P'][i]) < float(storage_dict['R'][i]):
            ws[f'P{i+4}'].font = Font(color = font_color,bold = bold_condition)
            ws[f'P{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

        elif float(storage_dict['P'][i]) >= float(storage_dict['R'][i]):
            ws[f'P{i+4}'].font = Font(color = None,bold = None)
            ws[f'P{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    if storage_dict['P'][i] in list_of_not_applicable:
      ws[f'P{i+4}'].font = Font(color = None,bold = None)
      ws[f'P{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING LIVE LOAD DEFLECTION CRITERIA
    if float(storage_dict['F'][i]) != 0 and float(storage_dict['Q'][i]) < design_roof_live_load_deflection_criteria:
      ws[f'Q{i+4}'].font = Font(color = font_color,bold = bold_condition)
      ws[f'Q{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['F'][i]) != 0 and float(storage_dict['Q'][i]) >= design_roof_live_load_deflection_criteria:
      ws[f'Q{i+4}'].font = Font(color = None,bold = None)
      ws[f'Q{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    if float(storage_dict['F'][i]) == 0 and float(storage_dict['Q'][i]) < design_floor_live_load_deflection_criteria:
      ws[f'Q{i+4}'].font = Font(color = font_color,bold = bold_condition)
      ws[f'Q{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['F'][i]) == 0 and float(storage_dict['Q'][i]) >= design_floor_live_load_deflection_criteria:
      ws[f'Q{i+4}'].font = Font(color = None,bold = None)
      ws[f'Q{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING TOTAL LOAD DEFLECTION CRITERIA
    if float(storage_dict['F'][i]) != 0 and float(storage_dict['R'][i]) < design_roof_total_load_deflection_criteria:
      ws[f'R{i+4}'].font = Font(color = font_color,bold = bold_condition)
      ws[f'R{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['F'][i]) != 0 and float(storage_dict['R'][i]) >= design_roof_total_load_deflection_criteria:
      ws[f'R{i+4}'].font = Font(color = None,bold = None)
      ws[f'R{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

    if float(storage_dict['F'][i]) == 0 and float(storage_dict['R'][i]) < design_floor_total_load_deflection_criteria:
      ws[f'R{i+4}'].font = Font(color = font_color,bold = bold_condition)
      ws[f'R{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif float(storage_dict['F'][i]) == 0 and float(storage_dict['R'][i]) >= design_floor_total_load_deflection_criteria:
      ws[f'R{i+4}'].font = Font(color = None,bold = None)
      ws[f'R{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

# ------------------------------------------------------------------------------------------------------------------------------

# CHECKING FOR WARNING
    if storage_dict['T'][i] != '-':
      ws[f'T{i+4}'].font = Font(color = font_color,bold = bold_condition)
      ws[f'T{i+4}'].fill = PatternFill(start_color = cell_color,end_color = cell_color,fill_type = filltype)

    elif storage_dict['T'][i] == '-':
      ws[f'T{i+4}'].font = Font(color = None,bold = None)
      ws[f'T{i+4}'].fill = PatternFill(start_color = None,end_color = None,fill_type = None)

In [36]:
wb.save(file_path)